# ALBERT BASE MODEL- TWEETS


# TRAIN : SET - 1

# VAL : 

> # SET - 2

> # SET - 3

> # test

In [18]:
!pip install -q ktrain

In [19]:
import ktrain
from ktrain import text 

In [20]:
import tensorflow as tf
import numpy as np 
import pandas as pd 

# Load data

In [21]:
set1 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set1.csv')
set2 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set2.csv')
set3 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set3.csv')

test = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/tweets_test_features.csv')

In [22]:
set1['target'] = set1['target'].apply(lambda x : 'disaster' if x==1 else 'not_a_disaster')
set2['target'] = set2['target'].apply(lambda x : 'disaster' if x==1 else 'not_a_disaster')

In [23]:
x_train = set1.text.values.tolist()
y_train = set1.target.values.tolist()

x_test = set2.text.values.tolist()
y_test = set2.target.values.tolist()

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# MODEL BUILDING

In [25]:
MODEL_NAME = 'albert-base-v2'

In [26]:
t = text.Transformer(MODEL_NAME, maxlen=500)

In [27]:
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)


preprocessing train...
language: en
train sequence lengths:
	mean : 15
	95percentile : 24
	99percentile : 27


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 15
	95percentile : 24
	99percentile : 27


In [28]:
model = t.get_classifier()



# Train: set- 1
# Val : set- 2

In [29]:
learner = ktrain.get_learner(model, train_data=trn,val_data=val, batch_size=6) #predictor = ktrain.get_predictor(learner.model, preproc=t)

In [30]:
learner.fit_onecycle(lr = 2e-5, epochs = 2 )



begin training using onecycle policy with max lr of 2e-05...
Train for 635 steps, validate for 60 steps
Epoch 1/2
635/635 [==============================] - 960s 2s/step - loss: 0.5552 - accuracy: 0.7178 - val_loss: 0.8031 - val_accuracy: 0.5707
Epoch 2/2
635/635 [==============================] - 934s 1s/step - loss: 0.6707 - accuracy: 0.5922 - val_loss: 0.6329 - val_accuracy: 0.7299


In [32]:
learner.fit_onecycle(lr = 2e-5, epochs = 2 )



begin training using onecycle policy with max lr of 2e-05...
Train for 635 steps, validate for 60 steps
Epoch 1/2
635/635 [==============================] - 932s 1s/step - loss: 0.5498 - accuracy: 0.7425 - val_loss: 0.5006 - val_accuracy: 0.7683
Epoch 2/2
635/635 [==============================] - 931s 1s/step - loss: 0.4567 - accuracy: 0.8048 - val_loss: 0.4161 - val_accuracy: 0.8150


In [36]:
learner.fit_onecycle(lr = 2e-5, epochs = 1 )



begin training using onecycle policy with max lr of 2e-05...
Train for 635 steps, validate for 60 steps
635/635 [==============================] - 932s 1s/step - loss: 0.4044 - accuracy: 0.8308 - val_loss: 0.4123 - val_accuracy: 0.8177


# F1 SCORE

In [37]:
learner.validate(class_names = ['Disaster','not_a_disaster'])

                precision    recall  f1-score   support

      Disaster       0.85      0.70      0.77       817
not_a_disaster       0.80      0.91      0.85      1086

      accuracy                           0.82      1903
     macro avg       0.83      0.80      0.81      1903
  weighted avg       0.82      0.82      0.81      1903



array([[570, 247],
       [100, 986]])

# Predict Proba

In [38]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [39]:
x2 = set2.text.values
x3 = set3.text.values
test_data = test.text.values

In [40]:
x2_proba_arr = predictor.predict(x2, return_proba=True)

In [41]:
x3_proba_arr = predictor.predict(x3, return_proba=True)

In [42]:
test_proba_arr = predictor.predict(test_data, return_proba=True)

In [43]:
x2_proba_list = x2_proba_arr.tolist()
x3_proba_list = x3_proba_arr.tolist()
test_proba_list = test_proba_arr.tolist()

In [51]:
disaster_probability = []
not_a_disaster_probability = []
for i in x2_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

x2_df = pd.DataFrame({'Albert_1': disaster_probability, 'Albert_0': not_a_disaster_probability})


In [52]:
disaster_probability = []
not_a_disaster_probability = []
for i in x3_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

x3_df = pd.DataFrame({'Albert_1': disaster_probability, 'Albert_0': not_a_disaster_probability})

In [53]:
disaster_probability = []
not_a_disaster_probability = []
for i in test_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

test_df = pd.DataFrame({'Albert_1': disaster_probability, 'Albert_0': not_a_disaster_probability})

# Save Probas to drive

In [54]:
x2_df.to_csv('set2_albert_proba.csv', index=False) 
!cp set2_albert_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

x3_df.to_csv('set3_albert_proba.csv', index=False) 
!cp set3_albert_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

test_df.to_csv('test_albert_proba.csv', index=False) 
!cp test_albert_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

# Sample of the files saved

In [55]:
x2_df[:2]

,Albert_1,Albert_0
0,0.097368,0.902632
1,0.973977,0.026023


In [56]:
x3_df[:2]

,Albert_1,Albert_0
0,0.130752,0.869248
1,0.290432,0.709568


In [57]:
test_df[:2]

,Albert_1,Albert_0
0,0.360784,0.639216
1,0.194359,0.805641
